# Using Ragas to Evaluate a RAG Application built with LangChain and LangGraph

In the following notebook, we'll be looking at how [Ragas](https://github.com/explodinggradients/ragas) can be helpful in a number of ways when looking to evaluate your RAG applications!

While this example is rooted in LangChain/LangGraph - Ragas is framework agnostic (you don't even need to be using a framework!).

- 🤝 Breakout Room #1
  1. Task 1: Installing Required Libraries
  2. Task 2: Set Environment Variables
  3. Task 3: Synthetic Dataset Generation for Evaluation using Ragas
  4. Task 4: Evaluating our Pipeline with Ragas
  5. Task 6: Making Adjustments and Re-Evaluating

But first! Let's set some dependencies!

## Dependencies and API Keys:

> NOTE: Please skip the pip install commands if you are running the notebook locally.

In [1]:
#!pip install -qU ragas==0.2.10

In [3]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

We'll also need to provide our API keys.

First, OpenAI's for our LLM/embedding model combination!

In [6]:
import os
from getpass import getpass
os.environ["OPENAI_API_KEY"] = getpass("Please enter your OpenAI API key!")

OPTIONALLY:

We can also provide a Ragas API key - which you can sign-up for [here](https://app.ragas.io/).

In [5]:
os.environ["RAGAS_APP_TOKEN"] = getpass("Please enter your Ragas API key!")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - and download our webpages which we'll be using for our data today.

These webpages are from [Simon Willison's](https://simonwillison.net/) yearly "AI learnings".

- [2023 Blog](https://simonwillison.net/2023/Dec/31/ai-in-2023/)
- [2024 Blog](https://simonwillison.net/2024/Dec/31/llms-in-2024/)

Let's start by collecting our data into a useful pile!

In [10]:
!mkdir data

In [11]:
!curl https://simonwillison.net/2023/Dec/31/ai-in-2023/ -o data/2023_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31287    0 31287    0     0    98k      0 --:--:-- --:--:-- --:--:--   98k


In [12]:
!curl https://simonwillison.net/2024/Dec/31/llms-in-2024/ -o data/2024_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 70146    0 70146    0     0   186k      0 --:--:-- --:--:-- --:--:--  186k


Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [13]:
from langchain_community.document_loaders import DirectoryLoader

path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [ ]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

In [15]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/26 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [16]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What happen with LLMs in 2023?,[Code may be the best application The ethics o...,"In 2023, it was discovered that Large Language...",single_hop_specifc_query_synthesizer
1,What are some challenges associated with using...,[Based Development As a computer scientist and...,"ChatGPT faces challenges such as gullibility, ...",single_hop_specifc_query_synthesizer
2,"What were the significant developments in AI, ...",[Simon Willison’s Weblog Subscribe Stuff we fi...,"In 2023, Large Language Models (LLMs) were rec...",single_hop_specifc_query_synthesizer
3,What Google do?,[easy to follow. The rest of the document incl...,A leaked Google document titled 'We Have No Mo...,single_hop_specifc_query_synthesizer
4,How have advancements in interactive applicati...,[<1-hop>\n\nPrompt driven app generation is a ...,Advancements in interactive applications have ...,multi_hop_abstract_query_synthesizer
5,How have advancements in interactive applicati...,[<1-hop>\n\nPrompt driven app generation is a ...,Advancements in interactive applications have ...,multi_hop_abstract_query_synthesizer
6,How have agents in AI and interactive applicat...,[<1-hop>\n\nPrompt driven app generation is a ...,"Agents in AI, despite being a frequently discu...",multi_hop_abstract_query_synthesizer
7,How have advancements in AI models contributed...,[<1-hop>\n\nPrompt driven app generation is a ...,Advancements in AI models have significantly c...,multi_hop_abstract_query_synthesizer
8,What advancements in GPT-4o and GPT-4 were obs...,[<1-hop>\n\nreasoning patterns. Another common...,"In 2024, significant advancements were observe...",multi_hop_specific_query_synthesizer
9,What are the cost and capabilities of Anthropi...,[<1-hop>\n\ngets you OpenAI’s most expensive m...,"Anthropic's Claude 3 Haiku, released in March ...",multi_hop_specific_query_synthesizer


#### OPTIONAL:

If you've provided your Ragas API key - you can use this web interface to look at the created data!

In [13]:
dataset.upload()

Testset uploaded! View at https://app.ragas.io/dashboard/alignment/testset/45d7742f-e0c6-4e85-9b66-6e819adfaec3


'https://app.ragas.io/dashboard/alignment/testset/45d7742f-e0c6-4e85-9b66-6e819adfaec3'

## LangChain RAG

Now we'll construct our LangChain RAG, which we will be evaluating using the above created test data!

### R - Retrieval

Let's start with building our retrieval pipeline, which will involve loading the same data we used to create our synthetic test set above.

> NOTE: We need to use the same data - as our test set is specifically designed for this data.

In [17]:
path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

Now that we have our data loaded, let's split it into chunks!

In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_documents = text_splitter.split_documents(docs)
len(split_documents)

73

#### ❓ Question: 

What is the purpose of the `chunk_overlap` parameter in the `RecursiveCharacterTextSplitter`?

Next up, we'll need to provide an embedding model that we can use to construct our vector store.

In [19]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

Now we can build our in memory QDrant vector store.

In [21]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

client = QdrantClient(":memory:")

client.create_collection(
    collection_name="ai_across_years",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="ai_across_years",
    embedding=embeddings,
)

We can now add our documents to our vector store.

In [22]:
_ = vector_store.add_documents(documents=split_documents)

Let's define our retriever.

In [23]:
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

Now we can produce a node for retrieval!

In [24]:
def retrieve(state):
  retrieved_docs = retriever.invoke(state["question"])
  return {"context" : retrieved_docs}

### Augmented

Let's create a simple RAG prompt!

In [25]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
You are a helpful assistant who answers questions based on provided context. You must only use the provided context, and cannot use your own knowledge.

### Question
{question}

### Context
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

### Generation

We'll also need an LLM to generate responses - we'll use `gpt-4o-mini` to avoid using the same model as our judge model.

In [26]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

Then we can create a `generate` node!

In [27]:
def generate(state):
  docs_content = "\n\n".join(doc.page_content for doc in state["context"])
  messages = rag_prompt.format_messages(question=state["question"], context=docs_content)
  response = llm.invoke(messages)
  return {"response" : response.content}

### Building RAG Graph with LangGraph

Let's create some state for our LangGraph RAG graph!

In [28]:
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.documents import Document

class State(TypedDict):
  question: str
  context: List[Document]
  response: str

Now we can build our simple graph!

> NOTE: We're using `add_sequence` since we will always move from retrieval to generation. This is essentially building a chain in LangGraph.

In [29]:
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

Let's do a test to make sure it's doing what we'd expect.

In [30]:
response = graph.invoke({"question" : "How are LLM agents useful?"})

In [31]:
response["response"]

'LLM agents can be useful in a couple of ways, primarily through their ability to act on behalf of users and to solve problems using various tools. They can function similarly to traditional agents, like travel agents, by helping users navigate tasks or make decisions. Additionally, LLMs can be enhanced with access to tools, allowing them to run processes in loops to address specific challenges.\n\nDespite some skepticism about their utility, LLMs are relatively easy to build, requiring only a few hundred lines of code and a substantial amount of quality training data. While training an LLM is not feasible for hobbyists due to costs, it is becoming more accessible than it once was.\n\nMoreover, LLMs can be run on personal devices, which has become increasingly possible thanks to advancements in model accessibility. However, the technology does warrant criticism due to concerns such as hallucination (producing information not aligned with reality), environmental impact, and ethical cons

## Evaluating the App with Ragas

Now we can finally do our evaluation!

We'll start by running the queries we generated usign SDG above through our application to get context and responses.

In [32]:
for test_row in dataset:
  response = graph.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]

In [33]:
dataset.to_pandas()

,user_input,retrieved_contexts,reference_contexts,response,reference,synthesizer_name
0,What happen with LLMs in 2023?,[This is Things we learned about LLMs in 2024 ...,[Code may be the best application The ethics o...,"In 2023, several significant developments occu...","In 2023, it was discovered that Large Language...",single_hop_specifc_query_synthesizer
1,What are some challenges associated with using...,"[But on the other hand, the things you sometim...",[Based Development As a computer scientist and...,Some challenges associated with using ChatGPT ...,"ChatGPT faces challenges such as gullibility, ...",single_hop_specifc_query_synthesizer
2,"What were the significant developments in AI, ...",[Simon Willison’s Weblog\n\nSubscribe\n\nStuff...,[Simon Willison’s Weblog Subscribe Stuff we fi...,"In 2023, significant developments in Large Lan...","In 2023, Large Language Models (LLMs) were rec...",single_hop_specifc_query_synthesizer
3,What Google do?,[Article Visitors Pageviews Bing: “I will not ...,[easy to follow. The rest of the document incl...,The provided context does not specifically def...,A leaked Google document titled 'We Have No Mo...,single_hop_specifc_query_synthesizer
4,How have advancements in interactive applicati...,[A lot of people are excited about AI agents—a...,[<1-hop>\n\nPrompt driven app generation is a ...,Advancements in interactive applications and t...,Advancements in interactive applications have ...,multi_hop_abstract_query_synthesizer
5,How have advancements in interactive applicati...,[A lot of people are excited about AI agents—a...,[<1-hop>\n\nPrompt driven app generation is a ...,Advancements in interactive applications and t...,Advancements in interactive applications have ...,multi_hop_abstract_query_synthesizer
6,How have agents in AI and interactive applicat...,[A lot of people are excited about AI agents—a...,[<1-hop>\n\nPrompt driven app generation is a ...,The evolution of agents in AI and interactive ...,"Agents in AI, despite being a frequently discu...",multi_hop_abstract_query_synthesizer
7,How have advancements in AI models contributed...,"[I think this means that, as individual users,...",[<1-hop>\n\nPrompt driven app generation is a ...,Advancements in AI models have significantly c...,Advancements in AI models have significantly c...,multi_hop_abstract_query_synthesizer
8,What advancements in GPT-4o and GPT-4 were obs...,[Training a GPT-4 beating model was a huge dea...,[<1-hop>\n\nreasoning patterns. Another common...,"In 2024, several advancements in GPT-4o and GP...","In 2024, significant advancements were observe...",multi_hop_specific_query_synthesizer
9,What are the cost and capabilities of Anthropi...,"[If you can gather the right data, and afford ...",[<1-hop>\n\ngets you OpenAI’s most expensive m...,"In 2024, Anthropic's Claude 3 series, particul...","Anthropic's Claude 3 Haiku, released in March ...",multi_hop_specific_query_synthesizer


Then we can convert that table into a `EvaluationDataset` which will make the process of evaluation smoother.

In [34]:
from ragas import EvaluationDataset

evaluation_dataset = EvaluationDataset.from_pandas(dataset.to_pandas())

We'll need to select a judge model - in this case we're using the same model that was used to generate our Synthetic Data.

In [35]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper

evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))

Next up - we simply evaluate on our desired metrics!

In [36]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, ResponseRelevancy, ContextEntityRecall, NoiseSensitivity
from ragas import evaluate, RunConfig

custom_run_config = RunConfig(timeout=360)

result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
result

Evaluating:   0%|          | 0/72 [00:00<?, ?it/s]

Exception raised in Job[41]: TimeoutError()
Exception raised in Job[53]: TimeoutError()
Exception raised in Job[65]: TimeoutError()
Exception raised in Job[71]: TimeoutError()


{'context_recall': 0.7610, 'faithfulness': 0.8682, 'factual_correctness': 0.4658, 'answer_relevancy': 0.8598, 'context_entity_recall': 0.5228, 'noise_sensitivity_relevant': 0.4327}

## Making Adjustments and Re-Evaluating

Now that we've got our baseline - let's make a change and see how the model improves or doesn't improve!

> NOTE: This will be using Cohere's Rerank model (which was updated fairly [recently](https://docs.cohere.com/v2/changelog/rerank-v3.5)) - please be sure to [sign-up for an API key!](https://docs.cohere.com/reference/about)

In [37]:
os.environ["COHERE_API_KEY"] = getpass("Please enter your Cohere API key!")

In [34]:
#!pip install -qU cohere langchain_cohere


We'll first set our retriever to return more documents, which will allow us to take advantage of the reranking.

In [38]:
retriever = vector_store.as_retriever(search_kwargs={"k": 20})

Reranking, or contextual compression, is a technique that uses a reranker to compress the retrieved documents into a smaller set of documents.

This is essentially a slower, more accurate form of semantic similarity that we use on a smaller subset of our documents.

In [42]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

def retrieve_adjusted(state):
  compressor = CohereRerank(model="rerank-v3.5")
  compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever, search_kwargs={"k": 5}
  )
  retrieved_docs = compression_retriever.invoke(state["question"])
  return {"context" : retrieved_docs}

We can simply rebuild our graph with the new retriever!

In [43]:
class State(TypedDict):
  question: str
  context: List[Document]
  response: str

graph_builder = StateGraph(State).add_sequence([retrieve_adjusted, generate])
graph_builder.add_edge(START, "retrieve_adjusted")
graph = graph_builder.compile()

In [44]:
response = graph.invoke({"question" : "How are LLM agents useful?"})
response["response"]

'LLM agents can be useful in certain contexts, particularly in writing code, as they demonstrate a strong capability in this area. The grammar rules of programming languages are less complex than natural languages, making it easier for LLMs to generate code effectively. However, their overall utility is questioned due to issues such as gullibility, where LLMs may struggle to distinguish truth from fiction. This skepticism is further amplified by the lack of real-world examples of LLM agents operating successfully in production environments, despite the excitement surrounding their potential. Thus, while LLM agents show promise, particularly in coding, concerns about their reliability and decision-making capabilities remain significant challenges that need to be addressed.'

In [45]:
import time

for test_row in dataset:
  response = graph.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]
  time.sleep(2) # To try to avoid rate limiting.

In [47]:
result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
result

Evaluating:   0%|          | 0/72 [00:00<?, ?it/s]

Exception raised in Job[65]: TimeoutError()
Exception raised in Job[71]: TimeoutError()


{'context_recall': 0.7610, 'faithfulness': 0.8309, 'factual_correctness': 0.4275, 'answer_relevancy': 0.8605, 'context_entity_recall': 0.5109, 'noise_sensitivity_relevant': 0.3846}

#### ❓ Question: 

Which system performed better, on what metrics, and why?